# 6r7m

In [1]:
using Printf, DelimitedFiles
using Plots
using DataFrames
using Revise, MDToolbox

In [2]:
function read_ascii(fname)
    lines = readlines(fname)
    natom = length(lines)
    xyz = zeros(Float64, 1, natom*3)
    for i = 1:natom
        xyz[1, 3*(i-1)+1] = parse(Float64, lines[i][31:38])
        xyz[1, 3*(i-1)+2] = parse(Float64, lines[i][39:46])
        xyz[1, 3*(i-1)+3] = parse(Float64, lines[i][47:54])
    end
    TrjArray(xyz=xyz)
end

read_ascii (generic function with 1 method)

In [3]:
fnames = []
omega = []
phi = []
prou = []
d = []
score = []
rmsd = []
struc = []

Any[]

In [4]:
nrun = 1000

1000

In [5]:
for irun = 1:nrun

    if irun % 100 == 0
        print("irun = $irun")
    end
    for i in 1:100
        filename = "$(irun)/complex.$(i).pdb"
        #t = mdload(filename)
        #t = MDToolbox.readpdb_biostructures(filename)
        t = read_ascii(filename)
        t1 = t[:, 1:1206]
        t2 = t[:, 1207:end]
        r = MDToolbox.compute_skrew(t1, t2)
        push!(fnames, filename)
        push!(omega, r.omega)
        push!(phi, r.phi / (2.0 * pi) * 360.0)
        push!(prou, r.prou)
        push!(d, r.d)
        push!(struc, t)
        #t_ca = t["atomname CA"]
        #push!(rmsd, compute_rmsd_permute(ref, t_ca, 2))
    end

    s = readdlm("$(irun)/zdock.out", skipstart=5)
    score = [score; s[1:100, end]]
end

irun = 100irun = 200irun = 300irun = 400irun = 500irun = 600irun = 700irun = 800irun = 900irun = 1000

In [6]:
df = DataFrame(fnames=fnames, score=score, omega=omega, phi=phi, prou=prou, d=d, struc=struc)

Row,fnames,score,omega,phi,prou,d,struc
,Any,Any,Any,Any,Any,Any,Any
1,1/complex.1.pdb,1440.86,"[0.507987, 0.151721, -0.847897]",22.4984,"[190.083, 181.186, 146.302]",3.36289,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 259.97 165.91 187.67 |"
2,1/complex.2.pdb,1355.91,"[-0.553359, -0.114757, -0.825]",177.56,"[60.9937, 152.925, -62.1826]",1.19473,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 225.29 182.16 215.38 |"
3,1/complex.3.pdb,1313.06,"[-0.518066, -0.21407, -0.828119]",30.4902,"[29.3759, 111.855, -47.292]",-0.369923,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 258.02 164.03 201.93 |"
4,1/complex.4.pdb,1254.79,"[-0.45877, -0.0508539, -0.887099]",179.398,"[100.897, 174.065, -62.1583]",-0.107229,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 221.55 181.18 213.08 |"
5,1/complex.5.pdb,1204.77,"[0.572324, 0.059546, 0.817863]",163.204,"[59.0539, 171.36, -53.8009]",2.91343,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 226.02 189.04 219.26 |"
6,1/complex.6.pdb,1198.03,"[-0.516551, 0.0168724, -0.85609]",165.107,"[86.4951, 194.415, -48.3581]",2.17879,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 223.57 177.00 210.79 |"
7,1/complex.7.pdb,1197.43,"[-0.0204815, -0.485007, -0.87427]",17.3521,"[165.279, 34.529, -23.0272]",12.8354,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 262.27 167.42 196.22 |"
8,1/complex.8.pdb,1196.01,"[-0.616462, -0.0343283, -0.786636]",164.788,"[45.6031, 179.582, -43.5745]",2.53024,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 227.36 179.05 214.09 |"
9,1/complex.9.pdb,1186.36,"[0.0707632, 0.954084, 0.29106]",156.273,"[206.179, -51.9118, 120.038]",19.5824,"1x2412 TrjArray{Float64, Int64}\n| 251.17 197.37 191.04 | … 197.52 222.74 192.30 |"


In [7]:
df_org = deepcopy(df);

In [8]:
#using BSON: @save, @load
#@save "skrew_parameters.bson" fnames score omega phi prou d df rmsd

In [9]:
#using BSON: @save, @load
#@save "df.bson" df

In [10]:
using JLD2

save("skrew_parameters.jld2", "fnames", fnames, "score", score, "omega", omega, "phi", phi, "prou", prou, "d", d, "df", df, "rmsd", rmsd)
save("df.jld2", "df", df)